In [1]:
# pip install beautifulsoup4
# pip install geopy

In [2]:
import gpxpy
import pandas as pd
import geopy
from geopy.geocoders import Nominatim

In [3]:
gpx = gpxpy.parse(open('20191005.gpx'))

print("{} track(s)".format(len(gpx.tracks)))
track = gpx.tracks[0]

print("{} segment(s)".format(len(track.segments)))
segment = track.segments[0]

print("{} point(s)".format(len(segment.points)))

1 track(s)
1 segment(s)
121 point(s)


In [4]:
data = []
segment_length = segment.length_3d()
for point_idx, point in enumerate(segment.points):
    data.append([point.longitude, point.latitude,
                 point.elevation, point.time, segment.get_speed(point_idx)])    


columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
df = pd.DataFrame(data, columns=columns)
df.head()

,Longitude,Latitude,Altitude,Time,Speed
0,-38.462279,-12.998631,NaN,2019-10-05 10:56:34.488000+00:00,3.672122
1,-38.460401,-12.999387,4.0,2019-10-05 10:57:35+00:00,6.352369
2,-38.464495,-12.996456,4.0,2019-10-05 10:58:36+00:00,7.958234
3,-38.464910,-12.992707,-4.0,2019-10-05 10:59:37+00:00,5.341736
4,-38.463733,-12.990971,6.0,2019-10-05 11:00:38+00:00,2.453005


In [5]:
geolocator = Nominatim(user_agent="britto_app")
geopy.geocoders.options.default_timeout = 10

In [7]:
x = df[['Latitude','Longitude']].head(20)

In [9]:
end = []
for index, row in x.iterrows():
    end.append(geolocator.reverse(row["Latitude"].astype(str)+','+row["Longitude"].astype(str)))

In [10]:
end

[Location(Benevento, Pituba, Salvador, SSA, Microrregião de Salvador, Região Metropolitana de Salvador, Mesorregião do Recôncavo baiano, BA, Região Nordeste, 41830595, Brasil, (-12.9987954259959, -38.46230862626, 0.0)),
 Location(Avenida Antônio Carlos Magalhães, Pituba, Salvador, SSA, Microrregião de Salvador, Região Metropolitana de Salvador, Mesorregião do Recôncavo baiano, BA, Região Nordeste, 41815320, Brasil, (-12.9993834162078, -38.4603992939335, 0.0)),
 Location(Subway, 656, Rua Anísio Teixeira, Itaigara, Salvador, SSA, Microrregião de Salvador, Região Metropolitana de Salvador, Mesorregião do Recôncavo baiano, BA, Região Nordeste, 41825-905, Brasil, (-12.9962176, -38.46409, 0.0)),
 Location(Rua Wanderley Pinho, Itaigara, Salvador, SSA, Microrregião de Salvador, Região Metropolitana de Salvador, Mesorregião do Recôncavo baiano, BA, Região Nordeste, 41815-070, Brasil, (-12.9926976915222, -38.4649104831841, 0.0)),
 Location(Rua Osvaldo Valente, Itaigara, Salvador, SSA, Microrregi

In [12]:
import requests
from bs4 import BeautifulSoup

In [13]:
url='https://www.gasnet.com.br/postos_gnv_lista.asp?nomestado=Bahia'
page = requests.get(url)

In [14]:
soup = BeautifulSoup(page.text, 'html.parser')

In [15]:
lista_post_scrp = []
for link in soup.find_all('a'):
    lista_post_scrp.append(link.get('href'))

In [16]:
reg = []
for lst in lista_post_scrp:
    reg.append(lst.split('&'))
#     for l in lst.split('&'):
#        print(l)

In [17]:
reg = reg[1:]

In [18]:
aux = []
for rg in reg:
    rg = rg[1:]
    aux.append(rg)


In [ ]:
df_postos = pd.DataFrame(aux)

In [27]:
linha = []
for a in aux:
    print('-- linha --')    
    linha.append(registro)
    registro = []
    for cmp in a:
        registro.append(cmp.split('=')[1])
        print(registro)
        
linha = linha[1:]

-- linha --
['Central Posto']
['Central Posto', 'BR']
['Central Posto', 'BR', 'Rodovia BR-101 Km 110.5 - s/nÂº,Alagoinhas,Bahia']
-- linha --
['Posto Gnc Alagoinhas']
['Posto Gnc Alagoinhas', 'BRANCA']
['Posto Gnc Alagoinhas', 'BRANCA', 'Av. Juracy MagalhÃ£es, s/nÂº,Alagoinhas,Bahia']
-- linha --
['Posto Guaraiuba']
['Posto Guaraiuba', 'BRANCA']
['Posto Guaraiuba', 'BRANCA', 'Via Parafuso, s/nÂº, Km 13,5,CamaÃ§ari,Bahia']
-- linha --
['Posto Interlagos']
['Posto Interlagos', 'BR']
['Posto Interlagos', 'BR', 'Rodovia BA-099, Estrada do CÃ´co, Km 21, s/nÂº,CamaÃ§ari,Bahia']
-- linha --
['Posto Radial B']
['Posto Radial B', 'BR']
['Posto Radial B', 'BR', 'Rua Manoel Alfredo MercÃªs, 783,CamaÃ§ari,Bahia']
-- linha --
['Posto Ondas']
['Posto Ondas', 'BR']
['Posto Ondas', 'BR', 'Estrada do CÃ´co, Km 22,CamaÃ§ari,Bahia']
-- linha --
['Posto Tvl 2']
['Posto Tvl 2', 'BRANCA']
['Posto Tvl 2', 'BRANCA', 'Rodovia BA-535, Km 15 (Via Parafuso),CamaÃ§ari,Bahia']
-- linha --
['Posto Trevo']
['Posto Tr

In [29]:
df_postos = pd.DataFrame(linha, columns=['nome','bandeira','endereco'])

In [31]:
df_postos.to_csv('postos.csv')